In [26]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize 

from keras import models
from keras import layers

from keras import optimizers

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

from keras.initializers import glorot_uniform  # Or your initializer of choice
import keras.backend as K
%matplotlib inline

In [27]:
# train_dir & validation_dir & test_dir 만들기

# 확인용 
train_dir = './data/zoom/'
validation_dir = './data/zoom/'

In [36]:
# 데이터 특징 추출 함수 corner harris 로 추출한 코너 값을 정규화 시킨다 -1 ~ 1로 변경된다.
def feature_extraction(img):
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    img_corner = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    img_gray = np.float32(img_gray) 

    result = cv2.cornerHarris(img_gray, 2, 3, 0.04)
    
    result_norm = normalize(result.reshape(-1, 1))
    result_norm = result_norm.reshape(150, 150, 1)
    return result_norm

In [37]:
# 모델 가중치 초기화 (평가 데이터 오버피팅 예방)
initial_weights = model.get_weights()

backend_name = K.backend()
if backend_name == 'tensorflow': 
    k_eval = lambda placeholder: placeholder.eval(session=K.get_session())
elif backend_name == 'theano': 
    k_eval = lambda placeholder: placeholder.eval()
else: 
    raise ValueError("Unsupported backend")

new_weights = [k_eval(glorot_uniform()(w.shape)) for w in initial_weights]

model.set_weights(new_weights)

In [41]:
# 케라스 모델 구축
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.rmsprop(lr=4e-4),
             metrics=['accuracy'])          
   

In [42]:
# 데이터 제너레이터 만들기 
train_datagen = ImageDataGenerator(
    width_shift_range=0.4,
    height_shift_range=0.4,
    preprocessing_function=feature_extraction
)

# train_datagen = ImageDataGenerator(preprocessing_function=feature_extraction)
test_datagen = ImageDataGenerator(preprocessing_function=feature_extraction)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150), 
    batch_size=20,
    class_mode='categorical',
)

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical',
)

Found 200 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


In [43]:
# 모델 훈련 & 평가
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=50,
)

Epoch 1/20
 10/100 [==>...........................] - ETA: 57s - loss: 2.3456 - acc: 0.1200

KeyboardInterrupt: 

In [ ]:
# 모델 훈련 & 평가 결과 플롯 그리기
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and Validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# 모델 저장
model.save('number_recoginition_draft_3.h5')